# LEAP3D Interface Demo

## Config

In [1]:
from leap3d.config import DATA_DIR, DATASET_DIR, X_MIN, X_MAX, Y_MIN, Y_MAX, Z_MIN, Z_MAX, SAFETY_OFFSET, MELTING_POINT, TIMESTEP_DURATION, SCALE_DISTANCE_BY, SCALE_TIME_BY, BASE_TEMPERATURE

## ScanParameters

In [23]:
from leap3d.scanning import ScanParameters

In [4]:
case_index = 0
params_filename =  DATA_DIR / "Params.npy"
rough_coord_filename =  DATA_DIR / "Rough_coord.npz"

In [24]:
case_params = ScanParameters(params_filename, rough_coord_filename, case_index,
                             x_min=X_MIN, x_max=X_MAX,
                             y_min=Y_MIN, y_max=Y_MAX,
                             z_min=Z_MIN, z_max=Z_MAX,
                             safety_offset=SAFETY_OFFSET,
                             melting_point=MELTING_POINT,
                             timestep_duration=TIMESTEP_DURATION,
                             scale_distance_by=SCALE_DISTANCE_BY,
                             scale_time_by=SCALE_TIME_BY)

Object bounds

In [ ]:
x_min, x_max, y_min, y_max, z_min, z_max = case_params.get_bounds()
x_min, x_max, y_min, y_max, z_min, z_max

Scanning bounds

In [ ]:
laser_x_min, laser_x_max, laser_y_min, laser_y_max = case_params.get_laser_bounds()
laser_x_min, laser_x_max, laser_y_min, laser_y_max

Edge points of current laser trajectory

In [ ]:
laser_x_min, laser_x_max, laser_y_min, laser_y_max = case_params.get_cross_section_scanning_bounds(laser_x=0, laser_y=0)
laser_x_min, laser_x_max, laser_y_min, laser_y_max

## ScanResults

In [6]:
from leap3d.scanning import ScanResults

In [7]:
case_index = 0
case_filename = DATA_DIR / "case_0000.npz"

In [8]:
case_results = ScanResults(case_filename)

In [ ]:
type(case_results.get_laser_data_at_timestep(0)[0])


In [ ]:
case_results.get_rough_temperatures_at_timestep(timestep=0).shape

In [ ]:
top_layer_temperatures = case_results.get_top_layer_temperatures(timestep=0)
top_layer_temperatures.shape

In [ ]:
laser_x, laser_y = case_results.get_laser_coordinates_at_timestep(timestep=0)
laser_x, laser_y

In [ ]:
coordinates, temperatures = case_results.get_coordinate_points_and_temperature_at_timestep(case_params, timestep=0)
coordinates, temperatures

In [27]:
import numpy as np

# top_layer_temperatures_t384 = case_results.get_top_layer_temperatures(timestep=384)
top_layer_temperatures_t385 = case_results.get_top_layer_temperatures(timestep=385)
top_layer_temperatures_t386 = case_results.get_top_layer_temperatures(timestep=386)

diff = top_layer_temperatures_t386 - top_layer_temperatures_t385
print(top_layer_temperatures_t386[diff > 10.0])
print(top_layer_temperatures_t385[diff > 10.0])
print(diff[diff > 10.0])
print(case_results.get_laser_data_at_timestep(386))



[ 682.1485   793.806    769.31335  753.5166   754.7451  1015.0751
 1810.3091  1638.9724  1002.7991   927.2842   883.2565   715.1585
 1167.0497   674.5829   766.9234   949.3082   767.2809   866.46716
  994.15564  822.59515]
[ 644.088    665.2476   705.98035  726.2364   741.15125  647.47034
  800.9297  1543.6045   980.65967  907.37646  868.77734  646.99646
  760.2031   663.28174  722.7192   903.50885  749.677    845.1901
  984.1435   811.24146]


TypeError: unhashable type: 'numpy.ndarray'

## Dataset

### Dataset preparation

In [2]:
from leap3d import dataset as dataset
from leap3d.dataset import check_if_prepared_data_exists

In [5]:
if not check_if_prepared_data_exists(DATASET_DIR / "dataset.hdf5"):
    dataset.prepare_raw_data(params_filename, rough_coord_filename, DATA_DIR, DATASET_DIR / "dataset.hdf5", cases=8)

['case_0000.npz', 'case_0001.npz', 'case_0002.npz', 'case_0003.npz', 'case_0004.npz', 'case_0005.npz', 'case_0006.npz', 'case_0007.npz', 'case_0008.npz', 'case_0009.npz'] 0000


KeyboardInterrupt: 

In [ ]:
if not check_if_prepared_data_exists(DATASET_DIR / "test_dataset.hdf5"):
    dataset.prepare_raw_data(params_filename, rough_coord_filename, DATA_DIR, DATASET_DIR / "test_dataset.hdf5", cases=[9,10])

### Dataset

In [ ]:
from leap3d.dataset import LEAP3DDataset

In [ ]:
dataset = LEAP3DDataset(DATASET_DIR, train=True)

In [ ]:
len(dataset)

In [ ]:
temperature_mean, temperature_variance, target_mean, target_variance = dataset.get_temperature_mean_and_variance(print_progress=True)

In [ ]:
print(temperature_mean, temperature_variance, target_mean, target_variance)

In [ ]:
from torchvision.transforms import transforms

from leap3d.transforms import normalize_temperature_2d


normalize_temperature = lambda x: normalize_temperature_2d(x, melting_point=MELTING_POINT, base_temperature=BASE_TEMPERATURE, inplace=True)

train_transforms = transforms.Compose([
        transforms.Lambda(normalize_temperature)
    ])

target_transforms = transforms.Compose([
    transforms.Lambda(normalize_temperature)
])
dataset = LEAP3DDataset(DATASET_DIR, train=True, transform=train_transforms, target_transform=target_transforms)

In [ ]:
normalized_temperature_mean, normalized_temperature_variance, normalized_target_mean, normalized_target_variance = dataset.get_temperature_mean_and_variance(print_progress=True)

In [ ]:
print(normalized_temperature_mean, normalized_temperature_variance, normalized_target_mean, normalized_target_variance)

In [ ]:
normalize_temperature = lambda x: normalize_temperature_2d(x, melting_point=MELTING_POINT, base_temperature=0, inplace=True)

train_transforms = transforms.Compose([
        transforms.Lambda(normalize_temperature)
    ])

target_transforms = transforms.Compose([
    transforms.Lambda(normalize_temperature)
])
dataset = LEAP3DDataset(DATASET_DIR, train=True, transform=train_transforms, target_transform=target_transforms)

In [ ]:
normalized_temperature_mean, normalized_temperature_variance, normalized_target_mean, normalized_target_variance = dataset.get_temperature_mean_and_variance(print_progress=True)

In [ ]:
print(normalized_temperature_mean, normalized_temperature_variance, normalized_target_mean, normalized_target_variance)

In [ ]:
x_train, extra_params, target = dataset[0]
x_train.shape, extra_params.shape, target.shape